In [ ]:
GCS:vertex_e2e_example/aiplatform-custom-training-2021-11-01-18:04:55.390/model/

In [4]:
import tensorflow as tf

In [12]:
!saved_model_cli show --dir "gs://vertex_e2e_example/aiplatform-custom-training-2021-11-01-18:04:55.390/model/"

The given SavedModel contains the following tag-sets:
'serve'


In [13]:
!saved_model_cli show --tag_set serve --dir "gs://vertex_e2e_example/aiplatform-custom-training-2021-11-01-18:04:55.390/model/"

The given SavedModel MetaGraphDef contains SignatureDefs with the following keys:
SignatureDef key: "__saved_model_init_op"
SignatureDef key: "serving_default"


In [14]:
!saved_model_cli show --tag_set serve --signature_def serving_default --dir "gs://vertex_e2e_example/aiplatform-custom-training-2021-11-01-18:04:55.390/model/"

The given SavedModel SignatureDef contains the following input(s):
  inputs['dropoff_latitude'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: serving_default_dropoff_latitude:0
  inputs['dropoff_longitude'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: serving_default_dropoff_longitude:0
  inputs['passenger_count'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: serving_default_passenger_count:0
  inputs['pickup_latitude'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: serving_default_pickup_latitude:0
  inputs['pickup_longitude'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: serving_default_pickup_longitude:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['output_1'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 1)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict


In [23]:
from google.cloud import aiplatform as aip

In [24]:
model = aip.Model.upload(
    display_name="test_keras",
    artifact_uri="gs://vertex_e2e_example/aiplatform-custom-training-2021-11-01-18:04:55.390/model/",
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-6:latest"
)

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/387138108602/locations/us-central1/models/7304702256153100288/operations/7250018114065137664
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/387138108602/locations/us-central1/models/7304702256153100288
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/387138108602/locations/us-central1/models/7304702256153100288')


In [9]:
!gsutil cat gs://vertex_e2e_taxi_data/taxifare/data/taxi-valid-000000000000.csv | head -2

2.5,2016-03-15 07:03:57 UTC,-73.901199340820312,40.85577392578125,-73.904006958007812,40.8521614074707,2,unused
2.5,2016-03-24 21:43:21 UTC,-73.990829467773438,40.717990875244141,-73.990562438964844,40.718437194824219,2,unused


model.batch_predict(
    job_display_name="test_predict",
    gcs_source="gs://vertex_e2e_taxi_data/taxifare/data/taxi-train-000000000000.csv",
    bigquery_destination_prefix="vf-grp-commercial-tst-explore.taxifare",
    machine_type="n1-standard-2"
)

### X above fails, which is expected (data has inputs plus labels, no headers as reqd by vertex for csv files). But vertex does not show any logs or stacktrace, only 'internal err'

### TODO move below code to common area (or common import) so that same code can be used for training and serving

In [8]:
# Defining the feature names into a list `CSV_COLUMNS`
CSV_COLUMNS = [
    #'fare_amount',
    'pickup_datetime',
    'pickup_longitude',
    'pickup_latitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'passenger_count',
    'key'
]
LABEL_COLUMN = 'fare_amount'
# Defining the default values into a list `DEFAULTS`
DEFAULTS = [['na'], [0.0], [0.0], [0.0], [0.0], [0.0], ['na']]
LABEL_DEFAULT = [0.0]
UNWANTED_COLS = ['pickup_datetime', 'key']


def features_and_labels(row_data, mode='eval'):
# The .pop() method will return item and drop from frame. 

    
    features = row_data
    
    for unwanted_col in UNWANTED_COLS:
        features.pop(unwanted_col)
	
    if mode == 'predict':
        return features
    else:
        label = row_data.pop(LABEL_COLUMN)
        return features, label


def create_dataset(pattern, batch_size=1, mode='eval'):
# The tf.data.experimental.make_csv_dataset() method reads CSV files into a dataset
    if mode == 'predict':
        dataset = tf.data.experimental.make_csv_dataset(pattern, batch_size, CSV_COLUMNS, DEFAULTS)
    
    else: 
    	dataset = tf.data.experimental.make_csv_dataset(pattern, batch_size, [LABEL_COLUMN]+CSV_COLUMNS, LABEL_DEFAULT+DEFAULTS)

# The map() function executes a specified function for each item in an iterable.
# The item is sent to the function as a parameter.
    dataset = dataset.map(lambda x: features_and_labels(x,mode))

    if mode == 'train':
# The shuffle() method takes a sequence (list, string, or tuple) and reorganize the order of the items.
        dataset = dataset.shuffle(buffer_size=1000).repeat()

    # take advantage of multi-threading; 1=AUTOTUNE
    dataset = dataset.prefetch(1)
    return dataset

In [111]:
test_ds = create_dataset('gs://vertex_e2e_taxi_data/taxifare/data/taxi-train-000000000000.csv')

In [9]:
pred_ds = create_dataset('gs://vertex_e2e_taxi_data/taxifare/data/taxi-pred-000000000000.csv', mode='predict')

In [7]:

import pprint
for data in pred_ds.take(2):
	pprint.pprint({k: v.numpy() for k,v in data.items()})
	
        #pprint({k:v.numpy() for k,v in data})
	print("\n")

2021-11-12 11:17:40.811329: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


{'dropoff_latitude': array([40.756546], dtype=float32),
 'dropoff_longitude': array([-73.98264], dtype=float32),
 'passenger_count': array([2.], dtype=float32),
 'pickup_latitude': array([40.75752], dtype=float32),
 'pickup_longitude': array([-73.97189], dtype=float32)}


{'dropoff_latitude': array([40.78193], dtype=float32),
 'dropoff_longitude': array([-73.95166], dtype=float32),
 'passenger_count': array([2.], dtype=float32),
 'pickup_latitude': array([40.740658], dtype=float32),
 'pickup_longitude': array([-73.98184], dtype=float32)}




In [117]:
import pprint
for data in test_ds.take(2):
	pprint.pprint({k: v.numpy() for k,v in data[0].items()})
	pprint.pprint(data[1])
        #pprint({k:v.numpy() for k,v in data})
	print("\n")

{'dropoff_latitude': array([40.82963], dtype=float32),
 'dropoff_longitude': array([-73.94097], dtype=float32),
 'passenger_count': array([2.], dtype=float32),
 'pickup_latitude': array([40.82564], dtype=float32),
 'pickup_longitude': array([-73.94363], dtype=float32)}
<tf.Tensor: shape=(1,), dtype=float32, numpy=array([3.], dtype=float32)>


{'dropoff_latitude': array([40.75132], dtype=float32),
 'dropoff_longitude': array([-73.99037], dtype=float32),
 'passenger_count': array([2.], dtype=float32),
 'pickup_latitude': array([40.7575], dtype=float32),
 'pickup_longitude': array([-73.99053], dtype=float32)}
<tf.Tensor: shape=(1,), dtype=float32, numpy=array([5.], dtype=float32)>




### why 2? expected behavior was to return size of the dataset

In [42]:
test_ds.cardinality().numpy()

-2

data for prediction -
1. will not have labels. can the training dataset handle that?
2. vertex model object can load csv with headers and create request. but data processing needs to happen on that data (since processing code is not part of model like in keras).
3. hence, prediction flow seems to be to xform data , save it in a BQ/GCS bucket. vertex model prediction will pick it up from there


|                |             |                        |
|   pred_raw     |             |     pred_processed     |
|________________|---xform---->|________________________|

In [ ]:
tf.data.experimental.save(pred_ds, 'gs://vertex_e2e_taxi_data/taxifare/data/pred_processed')

In [8]:
pred_ds

<PrefetchDataset shapes: OrderedDict([(pickup_longitude, (1,)), (pickup_latitude, (1,)), (dropoff_longitude, (1,)), (dropoff_latitude, (1,)), (passenger_count, (1,))]), types: OrderedDict([(pickup_longitude, tf.float32), (pickup_latitude, tf.float32), (dropoff_longitude, tf.float32), (dropoff_latitude, tf.float32), (passenger_count, tf.float32)])>

In [14]:
# https://www.tensorflow.org/api_docs/python/tf/data/experimental/CsvDataset#snapshot
pred_ds.snapshot(path='gs://vertex_e2e_taxi_data/taxifare/data/pred_processed.csv')

<MapDataset shapes: OrderedDict([(pickup_longitude, (1,)), (pickup_latitude, (1,)), (dropoff_longitude, (1,)), (dropoff_latitude, (1,)), (passenger_count, (1,))]), types: OrderedDict([(pickup_longitude, tf.float32), (pickup_latitude, tf.float32), (dropoff_longitude, tf.float32), (dropoff_latitude, tf.float32), (passenger_count, tf.float32)])>

In [12]:
pred_ds.snapshot(path='./')

<MapDataset shapes: OrderedDict([(pickup_longitude, (1,)), (pickup_latitude, (1,)), (dropoff_longitude, (1,)), (dropoff_latitude, (1,)), (passenger_count, (1,))]), types: OrderedDict([(pickup_longitude, tf.float32), (pickup_latitude, tf.float32), (dropoff_longitude, tf.float32), (dropoff_latitude, tf.float32), (passenger_count, tf.float32)])>

In [13]:
!ls

3_keras_sequential_api.ipynb	    6_replace_with_gcs_data.ipynb  taxi_trained
4_run_in_vertex.ipynb		    7_predict.ipynb		   train.py
5_export_data_from_bq_to_gcs.ipynb  data			   train_gcs.py


In [24]:
!touch header.yaml

In [ ]:
import tensorflow_io as tfio
tfio.experimental.serialization.save_dataset(
    pred_ds,'gs://vertex_e2e_taxi_data/taxifare/data/pred_processed.csv', './header.yamlb')

In [32]:
header = {}
for key in pred_ds.element_spec.keys():
    header[key] = {
        "shape": list(pred_ds.element_spec[key].shape),
        "dtype": pred_ds.element_spec[key].dtype,
    }
print(header)

{'pickup_longitude': {'shape': [1], 'dtype': tf.float32}, 'pickup_latitude': {'shape': [1], 'dtype': tf.float32}, 'dropoff_longitude': {'shape': [1], 'dtype': tf.float32}, 'dropoff_latitude': {'shape': [1], 'dtype': tf.float32}, 'passenger_count': {'shape': [1], 'dtype': tf.float32}}


In [38]:
features = {}
for x in pred_ds.take(2):
    # Each individual tensor is converted to a known serializable type.
    features = {key: value.numpy() for key, value in x.items()}
    pprint.pprint(features)  

{'dropoff_latitude': array([40.779636], dtype=float32),
 'dropoff_longitude': array([-73.957726], dtype=float32),
 'passenger_count': array([2.], dtype=float32),
 'pickup_latitude': array([40.756348], dtype=float32),
 'pickup_longitude': array([-73.97449], dtype=float32)}
{'dropoff_latitude': array([40.7513], dtype=float32),
 'dropoff_longitude': array([-73.97813], dtype=float32),
 'passenger_count': array([2.], dtype=float32),
 'pickup_latitude': array([40.74467], dtype=float32),
 'pickup_longitude': array([-73.981], dtype=float32)}


In [40]:
for item in pred_ds.take(1):
    print(item)

OrderedDict([('pickup_longitude', <tf.Tensor: shape=(1,), dtype=float32, numpy=array([-74.00917], dtype=float32)>), ('pickup_latitude', <tf.Tensor: shape=(1,), dtype=float32, numpy=array([40.7108], dtype=float32)>), ('dropoff_longitude', <tf.Tensor: shape=(1,), dtype=float32, numpy=array([-73.98931], dtype=float32)>), ('dropoff_latitude', <tf.Tensor: shape=(1,), dtype=float32, numpy=array([40.75371], dtype=float32)>), ('passenger_count', <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.], dtype=float32)>)])


In [10]:
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_example(row_dict):
  """
  Creates a tf.train.Example message ready to be written to a file.
  """
  # Create a dictionary mapping the feature name to the tf.train.Example-compatible
  # data type.
  #feature = {
  #    'feature0': _int64_feature(feature0),
  #    'feature1': _int64_feature(feature1),
  #    'feature2': _bytes_feature(feature2),
  #    'feature3': _float_feature(feature3),
  # }
  feature = {key: _float_feature(value.numpy()) for key, value in row_dict.items()}
  # Create a Features message using tf.train.Example.

  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

In [11]:
for item in pred_ds.take(1):
    ser_string = serialize_example(item)
    print(ser_string)
    example = tf.train.Example()
    example.ParseFromString(ser_string)
    print(example)

b'\n\x95\x01\n\x1b\n\x0fpassenger_count\x12\x08\x12\x06\n\x04\x00\x00\x00@\n\x1d\n\x11dropoff_longitude\x12\x08\x12\x06\n\x04\xbe\xeb\x93\xc2\n\x1c\n\x10dropoff_latitude\x12\x08\x12\x06\n\x04\xfc\x1a#B\n\x1c\n\x10pickup_longitude\x12\x08\x12\x06\n\x04\xc2\xf4\x93\xc2\n\x1b\n\x0fpickup_latitude\x12\x08\x12\x06\n\x04\xfe\x1b#B'
features {
  feature {
    key: "dropoff_latitude"
    value {
      float_list {
        value: 40.77635192871094
      }
    }
  }
  feature {
    key: "dropoff_longitude"
    value {
      float_list {
        value: -73.96043395996094
      }
    }
  }
  feature {
    key: "passenger_count"
    value {
      float_list {
        value: 2.0
      }
    }
  }
  feature {
    key: "pickup_latitude"
    value {
      float_list {
        value: 40.77733612060547
      }
    }
  }
  feature {
    key: "pickup_longitude"
    value {
      float_list {
        value: -73.97804260253906
      }
    }
  }
}



In [12]:
def generator():
  for features in pred_ds:
    yield serialize_example(features)

In [13]:
serialized_features_dataset = tf.data.Dataset.from_generator(
    generator, output_types=tf.string, output_shapes=())

In [14]:
serialized_features_dataset

<FlatMapDataset shapes: (), types: tf.string>

In [15]:
for item in serialized_features_dataset.take(1):
    print(item)

tf.Tensor(b'\n\x95\x01\n\x1c\n\x10pickup_longitude\x12\x08\x12\x06\n\x04~\xdd\x93\xc2\n\x1d\n\x11dropoff_longitude\x12\x08\x12\x06\n\x04\xb9\xd1\x93\xc2\n\x1c\n\x10dropoff_latitude\x12\x08\x12\x06\n\x04\x95\xb9"B\n\x1b\n\x0fpickup_latitude\x12\x08\x12\x06\n\x04\xfc\xc9"B\n\x1b\n\x0fpassenger_count\x12\x08\x12\x06\n\x04\x00\x00\x00@', shape=(), dtype=string)


## below took few hours to execute?
how can we make this faster, and run on vertex instead of nb cpu?

filename = 'pred.tfrecord'
writer = tf.data.experimental.TFRecordWriter(filename)
writer.write(serialized_features_dataset)

In [3]:
import tensorflow as tf
import numpy as np
filename = 'pred.tfrecord'
#read
raw_dataset = tf.data.TFRecordDataset(filename)
raw_dataset

for raw_record in raw_dataset.take(1):
  example = tf.train.Example()
  example.ParseFromString(raw_record.numpy())
  print(example)

  result = {}
  # example.features.feature is the dictionary
  for key, feature in example.features.feature.items():
    # The values are the Feature objects which contain a `kind` which contains:
    # one of three fields: bytes_list, float_list, int64_list

    kind = feature.WhichOneof('kind')
    result[key] = np.array(getattr(feature, kind).value)

  print(result)

features {
  feature {
    key: "dropoff_latitude"
    value {
      float_list {
        value: 40.78719711303711
      }
    }
  }
  feature {
    key: "dropoff_longitude"
    value {
      float_list {
        value: -73.9517822265625
      }
    }
  }
  feature {
    key: "passenger_count"
    value {
      float_list {
        value: 2.0
      }
    }
  }
  feature {
    key: "pickup_latitude"
    value {
      float_list {
        value: 40.72761535644531
      }
    }
  }
  feature {
    key: "pickup_longitude"
    value {
      float_list {
        value: -73.98524475097656
      }
    }
  }
}

{'pickup_longitude': array([-73.98524475]), 'passenger_count': array([2.]), 'dropoff_longitude': array([-73.95178223]), 'dropoff_latitude': array([40.78719711]), 'pickup_latitude': array([40.72761536])}


2021-11-12 22:35:42.276910: W tensorflow/core/data/root_dataset.cc:167] Optimization loop failed: Cancelled: Operation was cancelled


### try to make batch prediction using 1 example first

In [17]:
filename = 'pred_1.tfrecord' 
writer = tf.data.experimental.TFRecordWriter(filename) 
writer.write(serialized_features_dataset.take(1))

Instructions for updating:
To write TFRecords to disk, use `tf.io.TFRecordWriter`. To save and load the contents of a dataset, use `tf.data.experimental.save` and `tf.data.experimental.load`


In [20]:
!gsutil cp pred_1.tfrecord gs://vertex_e2e_taxi_data/taxifare/data/pred_results/

Copying file://pred_1.tfrecord [Content-Type=application/octet-stream]...
/ [1 files][  168.0 B/  168.0 B]                                                
Operation completed over 1 objects/168.0 B.                                      


In [28]:
model.batch_predict(
    job_display_name="batch-predict-1-instance",
    gcs_source="gs://vertex_e2e_taxi_data/taxifare/data/pred_results/pred_1.tfrecord",
    instances_format="tf-record",
    bigquery_destination_prefix="vf-grp-commercial-tst-explore.taxifare",
    machine_type="n1-standard-2"
)

INFO:google.cloud.aiplatform.jobs:Creating BatchPredictionJob
INFO:google.cloud.aiplatform.jobs:BatchPredictionJob created. Resource name: projects/387138108602/locations/us-central1/batchPredictionJobs/6585016920192319488
INFO:google.cloud.aiplatform.jobs:To use this BatchPredictionJob in another session:
INFO:google.cloud.aiplatform.jobs:bpj = aiplatform.BatchPredictionJob('projects/387138108602/locations/us-central1/batchPredictionJobs/6585016920192319488')
INFO:google.cloud.aiplatform.jobs:View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/6585016920192319488?project=387138108602
INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/387138108602/locations/us-central1/batchPredictionJobs/6585016920192319488 current state:
JobState.JOB_STATE_RUNNING
INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/387138108602/locations/us-central1/batchPredictionJobs/6585016920192319488 current state:
JobState.JOB_STAT

RuntimeError: Job failed with:
code: 13
message: "INTERNAL"
